In [1]:
# Importing the necessary libraries

In [2]:
import numpy as np
import pandas as pd
from gurobipy import *

In [3]:
# I. Cloud Computing

In [4]:
# Data Loading 
cpu_cap = 512
mem_cap = 1024
gpu_cap = 64

names = ['C1','C2','C3','M1','M2','M3','G1','G2','G3']

cpu = np.array([16, 32, 64, 8, 16, 32, 16, 32, 64])
mem = np.array([8, 16, 32, 32, 64, 128, 16, 32, 64])
gpu = np.array([1, 1, 1, 1, 1, 1, 2, 6, 8])

price = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])
rate = np.array([5.0, 5.0, 1.8, 3.0, 2.6, 1.0, 0.8, 0.4, 0.3])

T = 5

nInstances = len(cpu)
forecast = T * rate

In [5]:
# Question 1. Part 2
# Solving the simple model

In [6]:
# Creating the model
m = Model()

# Creating decision variables 
x = m.addVars(nInstances, lb = 0, ub = forecast)

# Creating the capacity constraints:
# CPU constraint
cpu_constr = m.addConstr(sum(cpu[i] * x[i] for i in range(nInstances)) <= cpu_cap)

# Memorty constraint 
mem_constr = m.addConstr(sum(mem[i] * x[i] for i in range(nInstances)) <= mem_cap)

# GPU constraint
gpu_constr = m.addConstr(sum(gpu[i] * x[i] for i in range(nInstances)) <= gpu_cap)

# Setting the objective function
m.setObjective(sum(price[i] * x[i] for i in range(nInstances)), GRB.MAXIMIZE)

# Update + solve:
m.update()
m.optimize()

Using license file /Users/tigranmargarian/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Model fingerprint: 0xc72eae64
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [2e+00, 2e+01]
  RHS range        [6e+01, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 8 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4400000e+03   7.200000e+01   0.000000e+00      0s
       3    1.0394286e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  1.039428571e+03


In [7]:
print(f'Optimized revenue equals ${round(m.objval,2)}')

Optimized revenue equals $1039.43


In [8]:
print('Requests accepted')
for i in range(nInstances):
    print(f'{names[i]}: {round(x[i].x,2)}')

Requests accepted
C1: 6.29
C2: 0.0
C3: 0.0
M1: 3.43
M2: 0.0
M3: 5.0
G1: 4.0
G2: 2.0
G3: 1.5


In [9]:
print(f'{round(gpu_constr.slack,2)} of GPU capacity is still available!')

17.29 of GPU capacity is still available!


In [10]:
print(f'That addition would bring '
      f'${32*cpu_constr.pi + 16*mem_constr.pi - 5} of revenue!') 

That addition would bring $9.0 of revenue!


In [11]:
# Question 1. Part 3
# Simulating current practice

In [12]:
# Preconditions:
# nSimulations = integer specifying number of simulations to run
# rates = array containing arrival rate (# / day) for each of the instance
# types (should be an array with 9 elements)
# T = length of horizon in days.

def generateArrivalSequences( nSimulations, rates, T ):
    total_rate = sum(rates)
    nTypes = len(rates)

    arrival_sequences_times = []
    arrival_sequences_types = [];

    for s in range(nSimulations):
        single_arrival_sequence_time = [];
        single_arrival_sequence_type = [];
        t = 0;
        while (t < T):
            single_time = np.random.exponential(1.0/total_rate)
            single_type = np.random.choice(nTypes, p= rates/total_rate )

            t += single_time;

            if (t < T):
                single_arrival_sequence_time.append(t)
                single_arrival_sequence_type.append(single_type)
            else:
                break

        arrival_sequences_times.append(np.array(single_arrival_sequence_time))
        arrival_sequences_types.append(np.array(single_arrival_sequence_type))
    return arrival_sequences_times, arrival_sequences_types



# # Code to test out above function
# np.random.seed(1)
# nSimulations_test = 100
# rates_test = np.array([5.0, 2.0, 3.0])
# T_test = 8
# x, y = generateArrivalSequences(nSimulations_test, rates_test, T_test)

# # If code above is working correctly, code below should show
# # value of 80.71:
# counts = np.array([len(y[i]) for i in range(nSimulations_test)] )
# counts.mean()

In [13]:
np.random.seed(10)
nSimulations = 100
arr_times,arr_types = generateArrivalSequences(nSimulations, rate, T)

In [14]:
ind_arrivals = np.zeros([nSimulations, nInstances])
for seq in range(nSimulations):
    for arrival in arr_types[seq]:
        ind_arrivals[seq][arrival] += 1

In [15]:
avg_arrivals = ind_arrivals.mean(axis=0)

In [16]:
for i in range(nInstances):
    print(f'The average number of arrivals of {names[i]} instance equals '
          f'{avg_arrivals[i]}')

The average number of arrivals of C1 instance equals 26.63
The average number of arrivals of C2 instance equals 24.38
The average number of arrivals of C3 instance equals 9.01
The average number of arrivals of M1 instance equals 14.88
The average number of arrivals of M2 instance equals 13.32
The average number of arrivals of M3 instance equals 4.93
The average number of arrivals of G1 instance equals 4.2
The average number of arrivals of G2 instance equals 2.27
The average number of arrivals of G3 instance equals 1.38


In [17]:
print(f'The average nubmer of arrivals for all types '
      f'is equal to {ind_arrivals.sum(axis=1).mean()}')

The average nubmer of arrivals for all types is equal to 101.0


In [18]:
print(f'The total forecast for all types is equal to {sum(forecast)}')

The total forecast for all types is equal to 99.5


In [19]:
print(f'The correlation between rates and average arrivals is '
      f'almost perfect')
print(np.corrcoef(rate,avg_arrivals))

The correlation between rates and average arrivals is almost perfect
[[1.         0.99797483]
 [0.99797483 1.        ]]


In [20]:
# Preconditions for code below:
# nSimulations = number of simulations to run
# nResources = number of different types of resources (= 3)
# B = numpy array of initial capacities of the resources
# arrival_sequences_times = array where each entry is arrival time sequence for that
# simulation
# arrival_sequences_types = array where each entry is sequence of request types for
# that simulation
nResources = 3
B = np.array([cpu_cap, mem_cap, gpu_cap])
# Note: code will not run; parts with ... need to be filled in.

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_capacity = np.zeros([nResources, nSimulations])

for s in range(nSimulations):
    b = B.copy();
    single_revenue = 0.0; # will contain the revenue of this simulation
    nArrivals = len(arr_times[s]);

    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (i)
        arrival_time = arr_times[s][j]
        i = arr_types[s][j]

        # Check if there is sufficient capacity for the request
        if ( b[0] >= cpu[i] and b[1] >= mem[i] and b[2] >= gpu[i]  ):
            # If there is sufficient capacity, accrue the revenue
            # and remove the capacity.
            single_revenue += price[i]
            b[0] -= cpu[i]
            b[1] -= mem[i]
            b[2] -= gpu[i]

    # Save the results of this simulation here ...
    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[:,s] = b

# Find the average revenue
mean_myopic_revenue = results_myopic_revenue.mean()
# Find the average remaining quantity of each resource
mean_myopic_remaining_capacity = results_myopic_remaining_capacity.mean(axis = 1)

In [21]:
print(f'The average revenue generated when following \n'
      f'the myopic strategy is ${mean_myopic_revenue}')

The average revenue generated when following 
the myopic strategy is $528.28


In [22]:
cap_names = ["CPU", "Memory", "GPU"]
for i in range(len(mean_myopic_remaining_capacity)):
    print(f'The remaining {cap_names[i]} capacity is '
          f'{mean_myopic_remaining_capacity[i]}')

The remaining CPU capacity is 0.24
The remaining Memory capacity is 371.52
The remaining GPU capacity is 37.42


In [23]:
# Question 1. Part 4
# A bid-price control policy

In [24]:
# Preconditions for code below:
# nSimulations = number of simulations to run
# nResources = number of different types of resources (= 3)
# B = numpy array of initial capacities of the resources
# arrival_sequences_times = array where each entry is arrival time sequence for that
# simulation
# arrival_sequences_types = array where each entry is sequence of request types for
# that simulation

# Your LP formulation should be defined here
# (or somewhere before the definition of bpc()...)
# ...

constrs = [cpu_constr,mem_constr,gpu_constr]
# Set the OutputFlag parameter to 0 to disable logging
m.Params.outputflag = 0

# As we did in-class, define a function bpc() to re-solve the LO each time:

def bpc(b, t):
    
#     for r in range(nResources):
#         # Set the RHS of the resource constraint to b[r] here
#         constrs[r].rhs = b[r]
    cpu_constr.rhs = b[0]
    mem_constr.rhs = b[1]
    gpu_constr.rhs = b[2]

    for i in range(nInstances):
        # Set the RHS of the forecast constraint for each instance
        # type to the expected number of requests over the duration
        # of the remaining horizon (T - t).
        x[i].ub = (T - t)*rate[i]

    # Re-solve the model:
    m.update()
    m.optimize()

    # Obtain the dual values/shadow prices
    dual_val = [cpu_constr.pi,mem_constr.pi,gpu_constr.pi]

    # Return the dual values:
    return dual_val

In [25]:
results_revenue = np.zeros(nSimulations)
results_remaining_capacity = np.zeros([nResources, nSimulations])
B = np.array([cpu_cap, mem_cap, gpu_cap])

for s in range(nSimulations):
    b = B.copy() #Initialize the current capacity
    single_revenue = 0.0 #Initialize the revenue garnered in this simulation
    nArrivals = len(arr_times[s])
    for j in range(nArrivals):
        # Take the next arrival time and type from the sequence
        arrival_time = arr_times[s][j]
        i = arr_types[s][j]

        # Check if there is enough capacity
        if (  b[0] >= cpu[i] and b[1] >= mem[i] and b[2] >= gpu[i] ):
            # Re-solve the LP and obtain the dual values
            dual_val = bpc(b, arrival_time)
            


            # Check if the revenue is at least the sum of the bid prices:
            if ( price[i] >= dual_val[0]*cpu[i] + dual_val[1]*mem[i] + dual_val[2]*gpu[i] ):
                # If there is sufficient capacity, accrue the revenue
                # and remove the capacity.
                single_revenue += price[i]
                b[0] -= cpu[i]
                b[1] -= mem[i]
                b[2] -= gpu[i]
   
    # Save the results of this simulation here:
    results_revenue[s] = single_revenue
    results_remaining_capacity[:,s] = b


# Find the average revenue:
mean_LP_revenue = results_revenue.mean()
# Find the average remaining quantity of each resource
mean_LP_remaining_capacity = results_remaining_capacity.mean(axis = 1)

In [26]:
print(f'The average revenue generated when following \n'
      f'the bid-price control strategy is ${mean_LP_revenue}')

The average revenue generated when following 
the bid-price control strategy is $925.59


In [27]:
for i in range(len(mean_LP_remaining_capacity)):
    print(f'The remaining {cap_names[i]} capacity is '
          f'{mean_LP_remaining_capacity[i]}')

The remaining CPU capacity is 27.2
The remaining Memory capacity is 4.88
The remaining GPU capacity is 20.62


In [28]:
# II. Designing a Sushi Menu

In [3]:
# Data Loading 
utilities_df = pd.read_csv("sushi_utilities_mat.csv", header=None)

In [4]:
info_df = pd.read_csv('sushi_info.csv', header=None)
info_df.columns = ['Type', 'Cat. Code', 'Price']

In [5]:
# Question 2. Part 1
# Understanding the data

In [6]:
# a) Customer 1 top-5 sushis

In [7]:
args = np.flip(np.argsort(utilities_df.iloc[0,:]))[:5].values

In [8]:
print(f'For customer 1 the top 5 sushis are:')
for i in range(5):
    print(f'Sushi type: {info_df.iloc[args[i],0]:<10}; '
          f'Type #: {args[i]+1:<3}; '
          f'Utility: {np.round(utilities_df.iloc[0,args[i]],3):<6}; '
          f'Category Code: {info_df.iloc[args[i],1]:<10}')


For customer 1 the top 5 sushis are:
Sushi type: negi_toro ; Type #: 38 ; Utility: 2.911 ; Category Code: 1         
Sushi type: ika       ; Type #: 4  ; Utility: 2.808 ; Category Code: 5         
Sushi type: maguro    ; Type #: 3  ; Utility: 2.806 ; Category Code: 1         
Sushi type: samon     ; Type #: 16 ; Utility: 2.772 ; Category Code: 1         
Sushi type: inari     ; Type #: 44 ; Utility: 2.751 ; Category Code: 11        


In [9]:
# b) Customer 2 worst 5 sushis

In [10]:
args = np.argsort(utilities_df.iloc[1,:])[:5].values

In [11]:
print(f'For customer 2 the worst 5 sushis are:')
for i in range(5):
    print(f'Sushi type: {info_df.iloc[args[i],0]:<15}; '
          f'Type #: {args[i]+1:<3}; '
          f'Utility: {np.round(utilities_df.iloc[1,args[i]],3):<6}; '
          f'Category Code: {info_df.iloc[args[i],1]:<10}')


For customer 2 the worst 5 sushis are:
Sushi type: karasumi       ; Type #: 94 ; Utility: -0.404; Category Code: 7         
Sushi type: komochi_konbu  ; Type #: 50 ; Utility: -0.317; Category Code: 7         
Sushi type: kyabia         ; Type #: 93 ; Utility: -0.258; Category Code: 7         
Sushi type: awabi          ; Type #: 15 ; Utility: -0.257; Category Code: 4         
Sushi type: kazunoko       ; Type #: 17 ; Utility: -0.231; Category Code: 7         


In [38]:
# c) best 5 sushis overall

In [39]:
# Ranking sushis for each customer
# Rank[i,j] == 1 if (j+1)-th sushi is the best for (i+1)-th customer
# Rank[i,j] == 100 if (j+1)-th sushi is the worst for (i+1)-th customer
ranks = np.zeros(utilities_df.shape)

for i in range(utilities_df.shape[0]):
    ranks[i,:] = np.argsort(np.flip(np.argsort(utilities_df.iloc[i,:]))) + 1

avg_ranks = ranks.mean(axis=0)
args = np.argsort(avg_ranks)[:5]

In [40]:
print(f'The best 5 sushis by average ranking are:')
for i in range(5):
    print(f'Sushi type: {info_df.iloc[args[i],0]:<15}; '
          f'Type #: {args[i]+1:<3}; '
          f'Avg. Ranking: {np.round(avg_ranks[args[i]],3):<6}; '
          f'Category Code: {info_df.iloc[args[i],1]:<10}')

The best 5 sushis by average ranking are:
Sushi type: negi_toro      ; Type #: 38 ; Avg. Ranking: 7.876 ; Category Code: 1         
Sushi type: maguro         ; Type #: 3  ; Avg. Ranking: 10.05 ; Category Code: 1         
Sushi type: kurumaebi      ; Type #: 80 ; Avg. Ranking: 10.084; Category Code: 6         
Sushi type: negi_toro_maki ; Type #: 62 ; Avg. Ranking: 10.214; Category Code: 1         
Sushi type: ebi            ; Type #: 1  ; Avg. Ranking: 11.932; Category Code: 6         


In [41]:
# d) the worst sushi

In [42]:
args = np.argsort(avg_ranks)[-1]

In [43]:
print(f'The worst sushi by average ranking is:\n')
print(f'Sushi type: {info_df.iloc[args,0]:<7}\n'
      f'Type #: {args+1:<3}\n'
      f'Avg. Ranking: {np.round(avg_ranks[args],3):<6}\n'
      f'Category Code: {info_df.iloc[args,1]:<10}')

The worst sushi by average ranking is:

Sushi type: kyabia 
Type #: 93 
Avg. Ranking: 90.648
Category Code: 7         


In [44]:
# e) the most controversial sushi

In [45]:
sds = ranks.std(axis=0)
args = np.argsort(sds)[-1]

In [46]:
print(f'The most controversial sushi by ranking SD is:\n')
print(f'Sushi type: {info_df.iloc[args,0]:<7} \n'
      f'Type #: {args+1:<3}\n'
      f'Ranking SD: {np.round(sds[args],3):<6}\n'
      f'Avg. Ranking: {np.round(avg_ranks[args],3):<6}\n'
      f'Category Code: {info_df.iloc[args,1]:<10}')

The most controversial sushi by ranking SD is:

Sushi type: awabi   
Type #: 15 
Ranking SD: 27.439
Avg. Ranking: 62.592
Category Code: 4         


In [47]:
# Question 2. Part 2
# Common-sense Solutions

In [48]:
# Adding no-choice information

In [12]:
nCustomers = utilities_df.shape[0]
nProducts = utilities_df.shape[1]

nonpurchase_utility = 3
nonpurchase_utilities = np.ones(utilities_df.shape[0])*nonpurchase_utility

In [13]:
utilities_df = pd.concat([utilities_df, pd.DataFrame(nonpurchase_utilities, 
                                                       columns=['100'])], 
                          axis=1)


In [14]:
info_df = info_df.append(pd.DataFrame(data=[['no choice', None, 0.0]], 
                                      columns=info_df.columns,
                                      index=[info_df.shape[0]]))

In [15]:
# Function to compute expected revenue
# S - the product line, represented as an array of integers
# Precondition: S is an array of integers containing only numbers between 0 and 99, with no repetitions.
def expected_profit(S):
    # Add the no-purchase option:
    # NB: the products are numbered from 0 to 99. Index 100 ( = nProducts) will correspond to the no-purchase option. 
    S2 = np.append(S, nProducts)
    
    choice_probabilities = {}
    for i in S2:
        choice_probabilities[i] = 0.0;
    
    for k in range(nCustomers):  
        ind = np.argmax( utilities_df.values[k,S2])
        choice_probabilities[ S2[ind] ] += 1.0/nCustomers
        
    exp_revenue = sum( [choice_probabilities[i] * info_df.iloc[i,2] for i in S])
        
    return exp_revenue, choice_probabilities

In [17]:
expected_profit( [0,1,2,3,4] )

(14.239620000000011,
 {0: 0.2700000000000002,
  1: 0.05200000000000003,
  2: 0.4200000000000003,
  3: 0.04200000000000002,
  4: 0.10000000000000007,
  100: 0.11600000000000009})

In [16]:
# Testing the function on product set of {1,2,3,4,5}
# Expected per-customer revenue is 14.2396
exp_profit, choice_probabilities = expected_profit( [0,1,2,3,4] )

In [54]:
print(f'Expected per-customer revenue equals ${np.round(exp_profit,2)}')

Expected per-customer revenue equals $14.24


In [55]:
# a) Offering everything

In [56]:
S_all = np.array([i for i in range(nProducts)])
exp_profit_all, choice_probabilities_all = expected_profit( S_all )
print(f"Expected profit (offer everything): ${round(exp_profit_all,2)}")

Expected profit (offer everything): $21.51


In [57]:
# b) Offering the 10 highest revenue sushis

In [58]:
S_mostrev = np.flip(np.argsort(info_df.values[:,2]))[:10]
for i in range(len(S_mostrev)):
    print(f'Sushi type: {info_df.iloc[S_mostrev[i],0]:<10}; '
          f'Price: {info_df.iloc[S_mostrev[i],2]:<10}')

Sushi type: toro      ; Price: 35.88     
Sushi type: awabi     ; Price: 29.17     
Sushi type: tarabagani; Price: 27.32     
Sushi type: akagai    ; Price: 26.66     
Sushi type: uni       ; Price: 26.3      
Sushi type: kurumaebi ; Price: 25.6      
Sushi type: chu_toro  ; Price: 25.34     
Sushi type: suzuki    ; Price: 25.07     
Sushi type: hirame    ; Price: 24.55     
Sushi type: botanebi  ; Price: 24.38     


In [59]:
exp_profit_mostrev, choice_probabilities_mostrev = expected_profit( S_mostrev )
print(f"Expected profit (offer most expensive): ${round(exp_profit_mostrev,2)}")

Expected profit (offer most expensive): $25.64


In [60]:
# c) Offering the most preferred sushi 

In [61]:
# Use numpy's function argmax to obtain the element with the highest value.
# Note: 0:nProducts will leave out column nProducts (= 100, which is where we are storing
# the no-purchase option.)
S_mostpref = np.argmax(utilities_df.values[:,0:nProducts], axis = 1)
# print(S_mostpref)
S_mostpref = np.unique(S_mostpref)

exp_profit_mostpref, choice_probabilities_mostpref = expected_profit( S_mostpref )
print(f"Expected profit (offer most preferred): ${round(exp_profit_mostpref,2)}")

Expected profit (offer most preferred): $21.51


In [62]:
# Question 2. Part 3
# An integer optimization model

In [63]:
# Implementation of the linear relaxation

In [64]:
m = Model()
m.Params.outputflag = 0
# Create the decision variables
x = m.addVars(nProducts, lb = 0, ub = 1)
y = m.addVars(nCustomers, nProducts+1, lb = 0, ub = 1)

# Create the constraints:
print("Creating constraints:")
for k in range(nCustomers):
    m.addConstr( sum(y[k,i] for i in range(nProducts+1)) == 1)
    for i in range(nProducts):
        m.addConstr( y[k,i] <= x[i] )
        m.addConstr( quicksum( utilities_df.values[k,j] * y[k,j] for j in range(nProducts+1)) >= utilities_df.values[k,i] * x[i] + utilities_df.values[k,nProducts]*(1 - x[i]))
    m.addConstr( quicksum( utilities_df.values[k,j] * y[k,j] for j in range(nProducts+1)) >= utilities_df.values[k,nProducts] )



# Create the objective:
print("Creating objective:")
m.setObjective( quicksum(info_df.iloc[i,2] * 1.0/nCustomers * y[k,i] for k in range(nCustomers) for i in range(nProducts)), GRB.MAXIMIZE)

# Update and solve
m.update()

print("Update completed")
m.optimize()

Creating constraints:
Creating objective:
Update completed


In [65]:
print(f'The optimized revenue from LP relaxation '
      f'equals to ${round(m.objval,2)}')

The optimized revenue from LP relaxation equals to $30.59


In [66]:
# The integer version of the LP

In [67]:
m = Model()
m.Params.outputflag = 0
# Create the decision variables
x = m.addVars(nProducts, vtype = GRB.BINARY)
y = m.addVars(nCustomers, nProducts+1, vtype = GRB.BINARY)

# Create the constraints:
print("Creating constraints:")
for k in range(nCustomers):
    m.addConstr( sum(y[k,i] for i in range(nProducts+1)) == 1)
    for i in range(nProducts):
        m.addConstr( y[k,i] <= x[i] )
        m.addConstr( quicksum( utilities_df.values[k,j] * y[k,j] for j in range(nProducts+1)) >= utilities_df.values[k,i] * x[i] + utilities_df.values[k,nProducts]*(1 - x[i]))
    m.addConstr( quicksum( utilities_df.values[k,j] * y[k,j] for j in range(nProducts+1)) >= utilities_df.values[k,nProducts] )



# Create the objective:
print("Creating objective:")
m.setObjective( quicksum(info_df.iloc[i,2] * 1.0/nCustomers * y[k,i] for k in range(nCustomers) for i in range(nProducts)), GRB.MAXIMIZE)

# Update and solve
m.update()

print("Update completed")
m.optimize()

Creating constraints:
Creating objective:
Update completed


In [68]:
print(f'The optimized revenue from integer LP '
      f'equals to ${round(m.objval,2)}')

The optimized revenue from integer LP equals to $26.24


In [69]:
menu = np.zeros(nProducts)
for i in range(nProducts):
    menu[i] = x[i].x

In [70]:
args = np.where(menu == 1.0)[0]

In [71]:
print(f'Sushis chosen for the menu:\n')
for i in range(len(args)):
    print(f'Sushi type: {info_df.iloc[args[i],0]:<15};  '
          f'Type #: {args[i]+1:<3};  '
          f'Price: {info_df.iloc[args[i],2]:<5};  '
          f'Avg. Ranking: {np.round(avg_ranks[args[i]],3):<6};  '
          f'Category Code: {info_df.iloc[args[i],1]:<10}')

Sushis chosen for the menu:

Sushi type: toro           ;  Type #: 9  ;  Price: 35.88;  Avg. Ranking: 16.52 ;  Category Code: 1         
Sushi type: awabi          ;  Type #: 15 ;  Price: 29.17;  Avg. Ranking: 62.592;  Category Code: 4         
Sushi type: saba           ;  Type #: 19 ;  Price: 11.86;  Avg. Ranking: 47.296;  Category Code: 0         
Sushi type: tarabagani     ;  Type #: 48 ;  Price: 27.32;  Avg. Ranking: 26.71 ;  Category Code: 6         
Sushi type: mentaiko_maki  ;  Type #: 73 ;  Price: 10.0 ;  Avg. Ranking: 60.736;  Category Code: 7         
Sushi type: ika_nattou     ;  Type #: 76 ;  Price: 8.73 ;  Avg. Ranking: 56.324;  Category Code: 5         
Sushi type: tobiuo         ;  Type #: 88 ;  Price: 12.0 ;  Avg. Ranking: 72.828;  Category Code: 0         


In [72]:
# Question 2. Part 4
# A constrained model

In [73]:
# Category variables (expluding non-purchase option)
cat = info_df['Cat. Code'][:100].astype(int)

In [74]:
# Number of represented categories (=12)
nCat = len(np.unique(cat))

In [75]:
# cats is a nProducts-by-nCat matrix where cats[i,c] == 1 
# if the i-th sushi belongs to the category c

cats = np.zeros([nProducts, nCat])
for i in range(nProducts):
    ind = cat[i]
    cats[i,ind] += 1

In [76]:
# category contraint 
for c in range(nCat):
    cat_constr = m.addConstr( sum( x[i]*cats[i][c] for i in range(nProducts)) >= 1)

In [77]:
m.update()
m.optimize()

In [78]:
print(f'The optimized revenue from constrained integer LP '
      f'equals to ${round(m.objval,2)}')

The optimized revenue from constrained integer LP equals to $25.88


In [79]:
menu_constr = np.zeros(nProducts)
for i in range(nProducts):
    menu_constr[i] = x[i].x

In [80]:
args = np.where(menu_constr == 1.0)[0]

In [81]:
print(f'Sushis chosen for the constrained menu:\n')
for i in range(len(args)):
    print(f'Sushi type: {info_df.iloc[args[i],0]:<15};  '
          f'Type #: {args[i]+1:<3};  '
          f'Price: {info_df.iloc[args[i],2]:<5};  '
          f'Avg. Ranking: {np.round(avg_ranks[args[i]],3):<6};  '
          f'Category Code: {info_df.iloc[args[i],1]:<10}')

Sushis chosen for the constrained menu:

Sushi type: tamago         ;  Type #: 8  ;  Price: 8.26 ;  Avg. Ranking: 42.29 ;  Category Code: 9         
Sushi type: toro           ;  Type #: 9  ;  Price: 35.88;  Avg. Ranking: 16.52 ;  Category Code: 1         
Sushi type: awabi          ;  Type #: 15 ;  Price: 29.17;  Avg. Ranking: 62.592;  Category Code: 4         
Sushi type: shako          ;  Type #: 18 ;  Price: 14.85;  Avg. Ranking: 63.808;  Category Code: 6         
Sushi type: saba           ;  Type #: 19 ;  Price: 11.86;  Avg. Ranking: 47.296;  Category Code: 0         
Sushi type: unagi          ;  Type #: 26 ;  Price: 17.9 ;  Avg. Ranking: 44.258;  Category Code: 3         
Sushi type: geso           ;  Type #: 31 ;  Price: 8.15 ;  Avg. Ranking: 60.886;  Category Code: 5         
Sushi type: tarabagani     ;  Type #: 48 ;  Price: 27.32;  Avg. Ranking: 26.71 ;  Category Code: 6         
Sushi type: ankimo         ;  Type #: 60 ;  Price: 14.0 ;  Avg. Ranking: 76.546;  Category Code